# training

> Training loop

In [ ]:
#|default_exp training

In [ ]:
#|hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from isaacai.utils import *
from isaacai.dataloaders import *
from isaacai.models import *

from datetime import datetime
import torchvision.transforms.functional as TF,torch.nn.functional as F

import matplotlib.pyplot as plt
import matplotlib as mpl
import fastcore.all as fc
import torch
from torch import nn, Tensor
from datasets import load_dataset, Dataset
from torch.utils.data import DataLoader
import pandas as pd , numpy as np
from torcheval.metrics import MulticlassAccuracy,Mean

In [ ]:
#|hide
torch.set_printoptions(precision=2, linewidth=140, sci_mode=False)
torch.manual_seed(1)
mpl.rcParams['image.cmap'] = 'gray'

import logging
logging.disable(logging.WARNING)

set_seed(42)
from IPython.display import clear_output

In [ ]:
xmean,xstd = 0.28, 0.35
@inplace
def transformi(b): b['image'] = [(TF.to_tensor(o)-xmean)/xstd for o in b['image']]

_dataset = load_dataset('fashion_mnist').with_transform(transformi)
_dataset = sample_dataset_dict(_dataset)
dls = DataLoaders.from_dataset_dict(_dataset, 64, num_workers=4)
clear_output()

In [ ]:
#| export
def run_callbacks(callbacks, method_name, trainer=None):
    for callback in sorted(callbacks, key=lambda x: getattr(x, 'order',0)):
        callback_method = getattr(callback, method_name,None)
        if callback_method is not None: callback_method(trainer)

In [ ]:
#| export
class MetricsCB:
    def __init__(self, precision=4, **metrics):
        self._precision=precision
        self.metrics = metrics
        self.metrics.update({'train_loss':Mean(),'valid_loss':Mean()})
        
    def _log(self,x): 
        _ff = fc.bind(round,ndigits=self._precision)
        for name in self.metrics.keys(): x[name]=_ff(x[name])
        x['elapsed'] = str(x['elapsed'])
        print(x)
    
    def before_batch(self,trainer): self.batch_size = len(trainer.batch[1])
    
    def after_batch(self,trainer):
        if trainer.training: 
            self.metrics['train_loss'].update(to_cpu(trainer.loss),weight=self.batch_size)
        else: 
            self.metrics['valid_loss'].update(to_cpu(trainer.loss),weight=self.batch_size)
            for name, metric in self.metrics.items():
                if name in ('train_loss','valid_loss'): continue
                self.metrics[name].update(to_cpu(trainer.preds),to_cpu(trainer.batch[1]))
            
    def before_epoch(self,trainer): self._st = datetime.now()
    def after_epoch(self,trainer):
        # compute metrics and append to epoch stats and display
        log = {name:float(metric.compute()) for name, metric in self.metrics.items()}    
        log['epoch'] = trainer.epoch
        log['elapsed'] = datetime.now() - self._st
        self._log(log)
        [metric.reset() for metric in self.metrics.values()]

In [ ]:
#| export
class DeviceCB:
    def __init__(self, device=def_device): fc.store_attr()
    def before_fit(self, trainer):
        if hasattr(trainer.model, 'to'): trainer.model.to(self.device)
    def before_batch(self, trainer): 
        trainer.batch = to_device(trainer.batch, device=self.device)

In [ ]:
#| export
def add_callbacks(trainer,callbacks):
    for callback in callbacks: 
        trainer.callbacks.append(callback.__class__.__name__)
        setattr(trainer,callback.__class__.__name__,callback)

In [ ]:
#| export
class BasicTrainCB:
    def predict(self,trainer): trainer.preds = trainer.model(trainer.batch[0])
    def get_loss(self,trainer): trainer.loss = trainer.loss_func(trainer.preds,trainer.batch[1])
    def backward(self,trainer): trainer.loss.backward()
    def step(self,trainer): trainer.opt.step()
    def zero_grad(self,trainer): trainer.opt.zero_grad()

In [ ]:
#| export 
class Trainer:
    def subclassing_method(self,**kwargs): pass

    @fc.delegates(to=subclassing_method)
    def __init__(self, dls, loss_func, opt_func, model, callbacks,**kwargs):
        self.callbacks = fc.L()
        self.add_callbacks(getattr(self,'base_callbacks',fc.L()))
        self.add_callbacks(callbacks)
        fc.store_attr(but='callbacks')
        self.subclassing_method(**kwargs)
        self.run_callbacks('after_init')
        
            
    def add_callbacks(self,callbacks): add_callbacks(self,callbacks)
    
    def one_batch(self):
        self.run_callbacks('before_batch')
        self.run_callbacks('predict')
        self.run_callbacks('get_loss')
        if self.training:
            self.run_callbacks('before_backward')
            self.run_callbacks('backward')
            self.run_callbacks('step')
            self.run_callbacks('zero_grad')
        self.run_callbacks('after_batch')

    def one_epoch(self):
        self.run_callbacks('before_epoch')
        self.model.train() # Training Loop
        self.run_callbacks('before_train')
        for self.batch in self.dls.train: self.one_batch()
        self.run_callbacks('after_train')
        self.model.eval()
        self.run_callbacks('before_valid')
        for self.batch in self.dls.valid: self.one_batch()
        self.run_callbacks('after_valid')
        self.run_callbacks('after_epoch')

    def fit(self, epochs=3, lr=1e-3):
        self.run_callbacks('before_fit')
        self.opt = self.opt_func(self.model.parameters(), lr)
        for self.epoch in range(epochs): self.one_epoch()
        self.run_callbacks('after_fit')

    @property
    def training(self): return self.model.training

    def run_callbacks(self,method_name): 
        cbs = [getattr(self,o) for o in self.callbacks]
        run_callbacks(cbs,method_name,self)

In [ ]:
trainer = Trainer(dls,
                  nn.CrossEntropyLoss(), 
                  torch.optim.Adam, 
                  SimpleNet(28*28,64,10), 
                  callbacks=[BasicTrainCB(),MetricsCB(accuracy=MulticlassAccuracy()), DeviceCB()])

In [ ]:
#| export
class MomentumTrainCB:
    def __init__(self,mom): fc.store_attr()
    def predict(self,trainer): trainer.preds = trainer.model(trainer.batch[0])
    def get_loss(self,trainer): trainer.loss = trainer.loss_func(trainer.preds,trainer.batch[1])
    def backward(self,trainer): trainer.loss.backward()
    def step(self,trainer): trainer.opt.step()
    def zero_grad(self,trainer): 
        with torch.no_grad():
            for p in trainer.model.parameters(): p.grad *= self.mom

In [ ]:
#| export
def InitDelegates(learner,method='subclassing_method'):
    learner.__init__ = fc.delegates(getattr(learner,method))(learner.__init__)
    return learner   

In [ ]:
#| export
@InitDelegates
class MomentumTrainer(Trainer): 
    def subclassing_method(self,precision=0.85): 
        self.add_callbacks([MomentumTrainCB(precision),DeviceCB()])

In [ ]:
trainer = MomentumTrainer(dls,
                  nn.CrossEntropyLoss(), 
                  torch.optim.SGD, 
                  SimpleNet(28*28,64,10), 
                  callbacks=[MetricsCB(Accuracy=MulticlassAccuracy())])

In [ ]:
trainer.fit()

{'Accuracy': 0.198, 'train_loss': 2.2719, 'valid_loss': 2.2044, 'epoch': 0, 'elapsed': '0:00:01.018725'}
{'Accuracy': 0.294, 'train_loss': 2.1569, 'valid_loss': 2.0836, 'epoch': 1, 'elapsed': '0:00:00.355708'}
{'Accuracy': 0.376, 'train_loss': 2.0255, 'valid_loss': 1.966, 'epoch': 2, 'elapsed': '0:00:00.349832'}


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()